In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px

from binary_classification import input_features

This is a simple implementation of linear regression exercise based on https://developers.google.com/machine-learning/crash-course/linear-regression.

The data is from https://data.cityofchicago.org/Transportation/Taxi-Trips-2013-2023-/wrvz-psew/about_data

In [226]:
# chicago_taxi_dataset = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/chicago_taxi_train.csv")
chicago_taxi_dataset = pd.read_csv("chicago_taxi_train.csv")

In [227]:
chicago_taxi_dataset.columns

Index(['TRIP_START_TIMESTAMP', 'TRIP_END_TIMESTAMP', 'TRIP_START_HOUR',
       'TRIP_SECONDS', 'TRIP_MILES', 'TRIP_SPEED', 'PICKUP_CENSUS_TRACT',
       'DROPOFF_CENSUS_TRACT', 'PICKUP_COMMUNITY_AREA',
       'DROPOFF_COMMUNITY_AREA', 'FARE', 'TIPS', 'TIP_RATE', 'TOLLS', 'EXTRAS',
       'TRIP_TOTAL', 'PAYMENT_TYPE', 'COMPANY'],
      dtype='object')

In [293]:
xi = chicago_taxi_dataset['TRIP_SECONDS'] / 60
yi_true = chicago_taxi_dataset['FARE']

In [294]:
fig = px.scatter(x=xi, y=yi_true,
                 labels={'x': 'x', 'y': 'y_true'},
                 title="Scatter of xi vs yi_true")
fig.show()

In [295]:
def update_parameters(b, w, eta, x, y_true):
    y_model = b + w * x

    slope_b = 2*np.mean(y_model - y_true)
    slope_w = 2*np.mean((y_model - y_true)*x)

    new_b = b - eta*slope_b
    new_w = w - eta*slope_w
    return new_b, new_w, slope_b, slope_w

def MSE(b, w, x, y_true):
    # x is a vector of length N
    y_model = b + w * x
    mse = np.mean((y_true - y_model)**2)
    return mse

def MAE(b, w, x, y_true):
    # x is a vector of length N
    y_model = b + w * x
    mae = np.mean((y_true - y_model))
    return mae


In [296]:
N_data = xi.size
N_epochs = 300
batch_size = 100

In [297]:
# standardize
mu_x, sx = np.mean(xi), np.std(xi)
mu_y, sy = np.mean(yi_true), np.std(yi_true)

xi_norm = (xi-mu_x) / sx
yi_true_norm = (yi_true-mu_y) / sy

In [298]:
b_i, w_i = 0, 1
step_size = 1.0e-4

all_MSE = np.zeros(N_epochs)
all_parameters = np.zeros((N_epochs, 2))
all_parameters_original = np.zeros((N_epochs, 2))

# 1) iterate over ALL epochs, each fully randomized
for j in range(N_epochs):
    perm = np.random.permutation(N_data)
    x_shuf = xi_norm[perm]
    y_shuf = yi_true_norm[perm]

    # 2) iterate over ALL batches, including a smaller tail batch
    for start in range(0, N_data, batch_size):
        end = start + batch_size
        xb = x_shuf[start:end]
        yb = y_shuf[start:end]
        b_i, w_i, _, _ = update_parameters(b_i, w_i, step_size, xb, yb)

    # compute loss in original units by denormalizing params
    w_orig = (sy / sx) * w_i
    b_orig = mu_y + sy * (b_i - w_i * (mu_x / sx))

    all_MSE[j] = MSE(b_orig, w_orig, xi, yi_true)

    all_parameters[j]          = [b_i, w_i]
    all_parameters_original[j] = [b_orig, w_orig]

print(all_MSE[-1])

89.44869459218685


In [299]:
fig = px.scatter(y=all_MSE,
                 labels={'x': 'x', 'y': 'MSE'},
                 title="Convergence of MSE")
fig.show()

In [300]:
all_parameters[-1]

array([1.02074778e-05, 8.30293914e-01])

In [301]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

epochs = np.arange(all_parameters_original.shape[0])  # 0, 1, 2, ..., N_epochs-1

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=("b parameter", "w parameter"))

# plot b
fig.add_trace(
    go.Scatter(x=epochs, y=all_parameters_original[:, 0], mode="lines", name="b"),
    row=1, col=1
)

# plot w
fig.add_trace(
    go.Scatter(x=epochs, y=all_parameters_original[:, 1], mode="lines", name="w"),
    row=2, col=1
)

fig.update_xaxes(title_text="Epoch", row=2, col=1)
fig.update_yaxes(title_text="b", row=1, col=1)
fig.update_yaxes(title_text="w", row=2, col=1)

fig.update_layout(height=600, width=800,
                  title_text="Parameters over epochs",
                  showlegend=False)

fig.show()


In [302]:
epochs = np.arange(all_parameters.shape[0])  # 0, 1, 2, ..., N_epochs-1

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=("b parameter", "w parameter"))

# plot b
fig.add_trace(
    go.Scatter(x=epochs, y=all_parameters[:, 0], mode="lines", name="b"),
    row=1, col=1
)

# plot w
fig.add_trace(
    go.Scatter(x=epochs, y=all_parameters[:, 1], mode="lines", name="w"),
    row=2, col=1
)

fig.update_xaxes(title_text="Epoch", row=2, col=1)
fig.update_yaxes(title_text="b", row=1, col=1)
fig.update_yaxes(title_text="w", row=2, col=1)

fig.update_layout(height=600, width=800,
                  title_text="Parameters over epochs",
                  showlegend=False)

fig.show()


# Now let's add more dimensionality to the parameters

In [303]:
def update_multidimensional_parameters(param_vec, eta, x, y_true):

    y_model = param_vec[0]

    for j in range(1, param_vec.size):
        y_model += param_vec[j] * x

    slope_0 = 2*np.mean(y_model - y_true)
    slope_w = 2*np.mean((y_model - y_true)*x)

    new_b = b - eta*slope_b
    new_w = w - eta*slope_w
    return new_b, new_w, slope_b, slope_w

def MSE(b, w, x, y_true):
    # x is a vector of length N
    y_model = b + w * x
    mse = np.mean((y_true - y_model)**2)
    return mse

def MAE(b, w, x, y_true):
    # x is a vector of length N
    y_model = b + w * x
    mae = np.mean((y_true - y_model))
    return mae


In [351]:
chicago_taxi_dataset.select_dtypes('number').columns

Index(['TRIP_START_HOUR', 'TRIP_SECONDS', 'TRIP_MILES', 'TRIP_SPEED',
       'PICKUP_CENSUS_TRACT', 'DROPOFF_CENSUS_TRACT', 'PICKUP_COMMUNITY_AREA',
       'DROPOFF_COMMUNITY_AREA', 'FARE', 'TIPS', 'TIP_RATE', 'TOLLS', 'EXTRAS',
       'TRIP_TOTAL'],
      dtype='object')

In [365]:
# Name of the features we'll train our model on.
input_features = [
    'TRIP_SECONDS',
    'TRIP_MILES',
    'TIP_RATE'
]

In [366]:
feature_mean = chicago_taxi_dataset.mean(numeric_only=True)
feature_std  = chicago_taxi_dataset.std(numeric_only=True)
numerical_features = chicago_taxi_dataset.select_dtypes('number').columns
normalized_dataset = (
    chicago_taxi_dataset[numerical_features] - feature_mean
) / feature_std

feature_numeical = chicago_taxi_dataset[input_features].to_numpy()
label_numeical   = chicago_taxi_dataset['FARE'].to_numpy()

In [367]:
def update_multidimensional_parameters(param_vec, eta, x, y_true):

    y_model = param_vec[0]
    for j in range(1, param_vec.size):
        y_model += param_vec[j] * x[:, j-1]

    param_vec_slope = np.zeros_like(param_vec)

    # Calculate the parameter slope
    param_vec_slope[0] = 2*np.mean(y_model - y_true)
    for j in range(1, param_vec.size):
        param_vec_slope[j] =  2*np.mean((y_model - y_true)*x[:, j-1])

    new_param_vec = param_vec - eta*param_vec_slope
    return new_param_vec

def multidimensional_MSE(param_vec, x, y_true):
    if hasattr(x, "to_numpy"):
        x = x.to_numpy()
    else:
        x = np.asarray(x)

    y_model = param_vec[0]
    for j in range(1, param_vec.size):
        y_model += param_vec[j] * x[:, j-1]

    mse = np.mean((y_true - y_model)**2)
    return mse

In [368]:
N_data = normalized_dataset[input_features].shape[0]
N_epochs = 150
batch_size = 100

In [369]:
step_size = 1.0e-4

all_MSE = np.zeros(N_epochs)

sigma_y = feature_std['FARE']
s_y     = feature_mean['FARE']

multi_parameter = np.zeros(len(input_features)+1)
# multi_parameter[0] = 0
# multi_parameter[1] = 1

# 1) iterate over ALL epochs, each fully randomized
for j in range(N_epochs):
    perm = np.random.permutation(N_data)
    x_shuf = normalized_dataset[input_features].to_numpy()[perm]
    y_shuf = normalized_dataset['FARE'].to_numpy()[perm]

    # # 2) iterate over ALL batches, including a smaller tail batch
    for start in range(0, N_data, batch_size):
        end = start + batch_size
        xb = x_shuf[start:end]
        yb = y_shuf[start:end]
        multi_parameter = update_multidimensional_parameters(multi_parameter, step_size, xb, yb)

    multi_parameter_original = np.copy(multi_parameter)

    # Calculate the original bias
    term = multi_parameter[0]
    for k in range(1, multi_parameter.size):
        term -= multi_parameter[k] * feature_mean[input_features].to_numpy()[k-1] / feature_std[input_features].to_numpy()[k-1]
    multi_parameter_original[0] = s_y + sigma_y * term

    for k in range(1, multi_parameter.size):
        multi_parameter_original[k] = sigma_y * multi_parameter[k] / feature_std[input_features].to_numpy()[k-1]

    all_MSE[j] = multidimensional_MSE(multi_parameter_original, feature_numeical, label_numeical)

print(all_MSE[-1])

12.353427139429897


In [370]:
fig = px.scatter(y=all_MSE,
                 labels={'x': 'x', 'y': 'MSE'},
                 title="Convergence of MSE")
fig.show()

In [371]:
multi_parameter_original

array([ 3.80945526e+00,  3.47945673e-03,  1.89460505e+00, -1.55506567e-02])